# DATA TRANSFORMATION - 12_Antec_Pat
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'Data/'

## 1. Load

In [2]:
modulo = 'Antecedentes_patologicos'

In [3]:
dcc = gf.diccionario_llaves()

In [4]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,EPS,fecha,FE_INICIO,FE_FIN,DS_OBSERVACIONES,Diagnostico,CodDiagnostico,year,month,year_month
0,653834,EPS/SURA,2010-10-10,2010-10-09,NaT,NaN,TRASPLANTE DE CORAZON,Z941,2010,10,201010
1,653834,EPS/SURA,2012-10-17,2012-10-17,NaT,NaN,OTRAS DERMATITIS ATOPICAS,L208,2012,10,201210
2,653834,EPS/SURA,2012-10-17,2012-10-17,NaT,NaN,"RINITIS ALERGICA, NO ESPECIFICADA",J304,2012,10,201210
3,653834,EPS/SURA,2017-08-01,2011-03-02,NaT,NaN,ASMA PREDOMINANTEMENTE ALERGICA,J450,2017,8,201708
4,653834,EPS/SURA,2012-10-17,2012-10-17,NaT,NaN,CHOQUE ANAFILACTICO DEBIDO A REACCION ADVERSA ...,T780,2012,10,201210


## 2. Transform

In [5]:
base = gf.letra_codigo( base, 'CodDiagnostico' )
base.head()

,id,EPS,fecha,FE_INICIO,FE_FIN,DS_OBSERVACIONES,Diagnostico,CodDiagnostico,year,month,year_month,CodDiagnostico_cod
0,653834,EPS/SURA,2010-10-10,2010-10-09,NaT,NaN,TRASPLANTE DE CORAZON,Z941,2010,10,201010,z
1,653834,EPS/SURA,2012-10-17,2012-10-17,NaT,NaN,OTRAS DERMATITIS ATOPICAS,L208,2012,10,201210,l
2,653834,EPS/SURA,2012-10-17,2012-10-17,NaT,NaN,"RINITIS ALERGICA, NO ESPECIFICADA",J304,2012,10,201210,j
3,653834,EPS/SURA,2017-08-01,2011-03-02,NaT,NaN,ASMA PREDOMINANTEMENTE ALERGICA,J450,2017,8,201708,j
4,653834,EPS/SURA,2012-10-17,2012-10-17,NaT,NaN,CHOQUE ANAFILACTICO DEBIDO A REACCION ADVERSA ...,T780,2012,10,201210,t


* Number of delivered doses per month
* Number of different diseases per month

In [8]:
base_ = base.groupby(['id','year','month','CodDiagnostico_cod'])["id"].count().\
    reset_index( name = 'ant' )
base_.head()

,id,year,month,CodDiagnostico_cod,ant
0,500547,2016,8,m,1
1,500547,2019,3,l,1
2,500588,2017,8,j,2
3,502989,2014,8,h,1
4,502989,2014,8,j,1


In [9]:
# all diagnoses different form j will be in the same category
base_['diag'] = base_['CodDiagnostico_cod'].apply( lambda x : 'j' if x == 'j' else 'otra' )

In [10]:
# pivot table to have separate variables, nans replaced with zero
base_p = base_.pivot_table(index=['id','year','month'], columns='diag', values=['ant'], aggfunc=np.sum).reset_index()
base_p.columns = ['_'.join(col).strip() for col in base_p.columns.values]
base_p.fillna( 0 , inplace = True)
base_p.head()

,id_,year_,month_,ant_j,ant_otra
0,500547,2016,8,0.0,1.0
1,500547,2019,3,0.0,1.0
2,500588,2017,8,2.0,0.0
3,502989,2014,8,1.0,1.0
4,506808,2013,5,2.0,1.0


In [11]:
base_p.rename( columns={ 'id_':'id', 'year_':'year', 'month_':'month' }, inplace=True)
base_p.head()

,id,year,month,ant_j,ant_otra
0,500547,2016,8,0.0,1.0
1,500547,2019,3,0.0,1.0
2,500588,2017,8,2.0,0.0
3,502989,2014,8,1.0,1.0
4,506808,2013,5,2.0,1.0


## 3. Merge with ids table

In [14]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_ant = ids_mensual.merge( base_p, how='left')
base_final_ant.head()

,id,year,month,ant_j,ant_otra
0,500547,2016,1,NaN,NaN
1,500547,2016,2,NaN,NaN
2,500547,2016,3,NaN,NaN
3,500547,2016,4,NaN,NaN
4,500547,2016,5,NaN,NaN


In [15]:
base_final_ant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53280 entries, 0 to 53279
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        53280 non-null  int64  
 1   year      53280 non-null  int64  
 2   month     53280 non-null  int64  
 3   ant_j     926 non-null    float64
 4   ant_otra  926 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 2.4 MB
